In [1]:
using DrWatson
@quickactivate

In [2]:
using BenchmarkTools, StaticArrays, Distributions
import ReverseDiff, Tracker, Zygote
const GRADS = (
    :ReverseDiff => ReverseDiff.gradient, 
    :Tracker => Tracker.gradient, 
    :Zygote => Zygote.gradient,
)

;

In [3]:
function trybtime(f, x; verbose=true)
    try
        t = @belapsed $f($x)
        verbose && println(" --- takes $t seconds")
        return t
    catch e
        verbose && println(" --- fails due to $e")
        return NaN
    end
end

function benchmark(f, x0; verbose=true)
    xs = [x0, SVector{length(x0), eltype(x0)}(x0)]
    res = []
    for x in xs
        verbose && print(rpad("$f(x::$(typeof(x)))", 55))
        trybtime(f, x; verbose=verbose)
        ts = []
        for grad in GRADS
            gradn, gradf = grad
            verbose && print("- $(lpad(gradn, 11)).gradient")
            if x isa SArray && gradn == :Zygote
                verbose && println(" --- skips for SArray because it never finishs")
                continue
            end
            t = trybtime(x -> gradf(f, x), x; verbose=verbose)
            push!(ts, t)
        end
        push!(res, ts)
    end
    return res
end

;

## Simple loop

In [4]:
function logdensity(x)
    res = 0
    for i = 1:size(x, 1)
        res += x[i]
    end
    return res
end

macro unroll_logdensity(n)
    ex = :(function $(Symbol("logdensity_$n"))(x) end)
    body = ex.args[2]
    push!(body.args, :(res = 0))
    for i in 1:n
        push!(body.args, :(res += x[$i]))
    end
    push!(body.args, :(return res))
    return ex
end

@macroexpand(@unroll_logdensity(10)) |> display

;

:(function Main.logdensity_10(var"#101#x")
      #= In[4]:10 =#
      var"#100#res" = 0
      var"#100#res" += var"#101#x"[1]
      var"#100#res" += var"#101#x"[2]
      var"#100#res" += var"#101#x"[3]
      var"#100#res" += var"#101#x"[4]
      var"#100#res" += var"#101#x"[5]
      var"#100#res" += var"#101#x"[6]
      var"#100#res" += var"#101#x"[7]
      var"#100#res" += var"#101#x"[8]
      var"#100#res" += var"#101#x"[9]
      var"#100#res" += var"#101#x"[10]
      return var"#100#res"
  end)

In [5]:
let x = rand(500), logdensity_unroll = @unroll_logdensity(500)
    benchmark(logdensity, x)
    benchmark(logdensity_unroll, x)
end

;

logdensity(x::Array{Float64,1})                         --- takes 5.635956284153006e-7 seconds
- ReverseDiff.gradient --- takes 0.000103103 seconds
-     Tracker.gradient --- takes 0.000291449 seconds
-      Zygote.gradient --- takes 0.001094527 seconds
logdensity(x::SArray{Tuple{500},Float64,1,500})         --- takes 4.541122448979592e-7 seconds
- ReverseDiff.gradient --- takes 0.000112217 seconds
-     Tracker.gradient --- fails due to ErrorException("setindex!(::SArray{Tuple{500},Float64,1,500}, value, ::Int) is not defined.")
-      Zygote.gradient --- skips for SArray because it never finishs
logdensity_500(x::Array{Float64,1})                     --- takes 5.295185185185184e-7 seconds
- ReverseDiff.gradient --- takes 0.000128364 seconds
-     Tracker.gradient --- takes 0.00031313 seconds
-      Zygote.gradient --- takes 0.004269467 seconds
logdensity_500(x::SArray{Tuple{500},Float64,1,500})     --- takes 3.6e-11 seconds
- ReverseDiff.gradient --- takes 0.000132293 seconds
-     T

In [6]:
let x = rand(1_000), logdensity_unroll = @unroll_logdensity(1_000)
    benchmark(logdensity, x)
    benchmark(logdensity_unroll, x)
end

;

logdensity(x::Array{Float64,1})                         --- takes 1.383e-6 seconds
- ReverseDiff.gradient --- takes 0.000221063 seconds
-     Tracker.gradient --- takes 0.001030266 seconds
-      Zygote.gradient --- takes 0.003765213 seconds
logdensity(x::SArray{Tuple{1000},Float64,1,1000})       --- takes 1.0686153846153846e-6 seconds
- ReverseDiff.gradient --- takes 0.000218738 seconds
-     Tracker.gradient --- fails due to ErrorException("setindex!(::SArray{Tuple{1000},Float64,1,1000}, value, ::Int) is not defined.")
-      Zygote.gradient --- skips for SArray because it never finishs
logdensity_1000(x::Array{Float64,1})                    --- takes 1.0477000000000002e-6 seconds
- ReverseDiff.gradient --- takes 0.000274715 seconds
-     Tracker.gradient --- takes 0.000998024 seconds
-      Zygote.gradient --- takes 0.018179059 seconds
logdensity_1000(x::SArray{Tuple{1000},Float64,1,1000})  --- takes 2.5e-11 seconds
- ReverseDiff.gradient --- takes 0.000262579 seconds
-     Tracker.

## Stochastic volatility

In [7]:
# @model sto_volatility(y, ::Type{Tv}=Vector{Float64}) where {Tv} = begin
#     T = length(y)
#     ϕ ~ Uniform(-1, 1)
#     σ ~ truncated(Cauchy(0, 5), 0, Inf)
#     μ ~ Cauchy(0, 10)
#     h = Tv(undef, T)
#     h[1] ~ Normal(μ, σ / sqrt(1 - ϕ^2))
#     for t in 2:T
#         h[t] ~ Normal(μ + ϕ * (h[t-1] - μ), σ)
#     end
#     y ~ ArrayDist(Normal.(0, exp.(h / 2)))
# end

function rand_sv(T)
    ϕ = rand(Uniform(-1, 1))
    σ = rand(truncated(Cauchy(0, 5), 0, Inf))
    μ = rand(Cauchy(0, 10))
    h = zeros(T)
    h[1] = rand(Normal(μ, σ / sqrt(1 - ϕ^2)))
    for t in 2:T
        h[t] = rand(Normal(μ + ϕ * (h[t-1] - μ), σ))
    end
    y = rand.(Normal.(0, exp.(h / 2)))
    theta = [ϕ, σ, μ, h...]
    return (theta=theta, y=y)
end

function get_logdensity_sv(y)
    function logdensity_sv(theta)
        res = 0
        T = length(y)
        ϕ = theta[1]
        res += logpdf(Uniform(-1, 1), ϕ)
        σ = theta[2]
        res += logpdf(truncated(Cauchy(0, 5), 0, Inf), σ)
        μ = theta[3]
        res += logpdf(Cauchy(0, 10), μ)
        h = Vector{eltype(theta)}(undef, T)
        h[1] = theta[4]
        res += logpdf(Normal(μ, σ / sqrt(1 - ϕ^2)), h[1])
        for t in 2:T
            h[t] = theta[t+3]
            res += logpdf(Normal(μ + ϕ * (h[t-1] - μ), σ), h[t])
        end
        res += sum(logpdf.(Normal.(0, exp.(h / 2)), y))
        return res
    end
    return logdensity_sv
end

macro get_logdensity_unroll_sv(T)
    fn = Symbol("logdensity_sv_$T")
    getfn = Symbol("get_logdensity_sv_$T")
    ex = :(function $fn(theta) end)
    body = ex.args[2]
    push!(body.args, :(res = 0))
    push!(body.args, :(T = length(y)))
    push!(body.args, :(ϕ = theta[1]))
    push!(body.args, :(res += logpdf(Uniform(-1, 1), ϕ)))
    push!(body.args, :(σ = theta[2]))
    push!(body.args, :(res += logpdf(truncated(Cauchy(0, 5), 0, Inf), σ)))
    push!(body.args, :(μ = theta[3]))
    push!(body.args, :(res += logpdf(Cauchy(0, 10), μ)))
    push!(body.args, :(h = Vector{eltype(theta)}(undef, $T)))
    push!(body.args, :(h[1] = theta[4]))
    push!(body.args, :(res += logpdf(Normal(μ, σ / sqrt(1 - ϕ^2)), h[1])))
    for t in 2:T
        push!(body.args, :(h[$t] = theta[$t+3]))
        push!(body.args, :(res += logpdf(Normal(μ + ϕ * (h[$t-1] - μ), σ), h[$t])))
    end
    push!(body.args, :(res += sum(logpdf.(Normal.(0, exp.(h / 2)), y))))
    push!(body.args, :(return res))
    return :(function $getfn(y) $ex; return $fn end)
end

@macroexpand(@get_logdensity_unroll_sv(5)) |> display

;

:(function Main.get_logdensity_sv_5(var"#128#y")
      #= In[7]:73 =#
      function var"#127#logdensity_sv_5"(var"#135#theta")
          #= In[7]:54 =#
          var"#129#res" = 0
          var"#130#T" = Main.length(var"#128#y")
          var"#131#ϕ" = var"#135#theta"[1]
          var"#129#res" += Main.logpdf(Main.Uniform(-1, 1), var"#131#ϕ")
          var"#132#σ" = var"#135#theta"[2]
          var"#129#res" += Main.logpdf(Main.truncated(Main.Cauchy(0, 5), 0, Main.Inf), var"#132#σ")
          var"#133#μ" = var"#135#theta"[3]
          var"#129#res" += Main.logpdf(Main.Cauchy(0, 10), var"#133#μ")
          var"#134#h" = Main.Vector{Main.eltype(var"#135#theta")}(Main.undef, 5)
          var"#134#h"[1] = var"#135#theta"[4]
          var"#129#res" += Main.logpdf(Main.Normal(var"#133#μ", var"#132#σ" / Main.sqrt(1 - var"#131#ϕ" ^ 2)), var"#134#h"[1])
          var"#134#h"[2] = var"#135#theta"[2 + 3]
          var"#129#res" += Main.logpdf(Main.Normal(var"#133#μ" + var"#131#ϕ" * (var"#134#h"[

In [8]:
let res = rand_sv(50), get_logdensity_unroll_sv = @get_logdensity_unroll_sv(50)
    logdensity_sv = get_logdensity_sv(res.y)
    logdensity_sv_unroll = get_logdensity_unroll_sv(res.y)
    
    benchmark(logdensity_sv, res.theta)
    benchmark(logdensity_sv_unroll, res.theta)
end

;

logdensity_sv(x::Array{Float64,1})                      --- takes 1.8761e-6 seconds
- ReverseDiff.gradient --- takes 0.000254523 seconds
-     Tracker.gradient --- takes 0.000194237 seconds
-      Zygote.gradient --- fails due to MethodError(Irrational{:log2π}, (1,), 0x00000000000069ee)
logdensity_sv(x::SArray{Tuple{53},Float64,1,53})        --- takes 1.9711e-6 seconds
- ReverseDiff.gradient --- takes 0.000250608 seconds
-     Tracker.gradient --- fails due to ErrorException("setindex!(::SArray{Tuple{53},Float64,1,53}, value, ::Int) is not defined.")
-      Zygote.gradient --- skips for SArray because it never finishs
#136#logdensity_sv_50(x::Array{Float64,1})              --- takes 1.9462e-6 seconds
- ReverseDiff.gradient --- takes 0.000263125 seconds
-     Tracker.gradient --- takes 0.00019578 seconds
-      Zygote.gradient --- fails due to MethodError(Irrational{:log2π}, (1,), 0x0000000000006a1f)
#136#logdensity_sv_50(x::SArray{Tuple{53},Float64,1,53}) --- takes 1.8994000000000002e-

In [9]:
let res = rand_sv(500), get_logdensity_unroll_sv = @get_logdensity_unroll_sv(500)
    logdensity_sv = get_logdensity_sv(res.y)
    logdensity_sv_unroll = get_logdensity_unroll_sv(res.y)
    
    benchmark(logdensity_sv, res.theta)
    benchmark(logdensity_sv_unroll, res.theta)
end

;

logdensity_sv(x::Array{Float64,1})                      --- takes 1.7029e-5 seconds
- ReverseDiff.gradient --- takes 0.002507677 seconds
-     Tracker.gradient --- takes 0.002374453 seconds
-      Zygote.gradient --- fails due to MethodError(Irrational{:log2π}, (1,), 0x0000000000006a52)
logdensity_sv(x::SArray{Tuple{503},Float64,1,503})      --- takes 1.7351e-5 seconds
- ReverseDiff.gradient --- takes 0.00247022 seconds
-     Tracker.gradient --- fails due to ErrorException("setindex!(::SArray{Tuple{503},Float64,1,503}, value, ::Int) is not defined.")
-      Zygote.gradient --- skips for SArray because it never finishs
#168#logdensity_sv_500(x::Array{Float64,1})             --- takes 1.88e-5 seconds
- ReverseDiff.gradient --- takes 0.002772974 seconds
-     Tracker.gradient --- takes 0.002312652 seconds
-      Zygote.gradient --- fails due to MethodError(Irrational{:log2π}, (1,), 0x0000000000006a83)
#168#logdensity_sv_500(x::SArray{Tuple{503},Float64,1,503}) --- takes 1.696e-5 seconds
